In [1]:
import pandas as pd

In [3]:
cols = ('Cat','Year','Round','Permanent_Tournament_#','Course_#','Hole','Start_X_Coordinate',
        'Start_Y_Coordinate','Distance_from_hole','Strokes_Gained','Time','Par_Value','Player_#')
data = pd.concat([pd.read_csv('../data/%d.csv' % year,usecols=cols) for year in range(2003,2004)])

In [4]:
import pandas as pd
import numpy as np
from scipy.sparse import csc_matrix
from scipy.spatial.distance import pdist,squareform
import math,os,sys,multiprocessing,gc,pickle,itertools
from collections import defaultdict

In [5]:
def get_matrix(tournament,conditon,sig_p):
    arr,arr1 = np.zeros((n_players,n_players)),np.zeros((n_players,n_players))
    for (round,course,hole),df in data[data.tourn_num==tournament].groupby(['Round','Course_','Hole']):
        subset = df.query(condition)[['Start_X_Coordinate','Start_Y_Coordinate','Distance_from_hole',
                                      'Strokes_Gained','Time','Player_Index']].values
        num_shots = subset.shape[0]
        dists = squareform(pdist(subset[:,0:2]))
        w_1 = w_1 = 1/(dists/(np.add.outer(subset[:,2],subset[:,2])/2) + .01)**e_d
        w_2 = 1/((np.abs(np.subtract.outer(subset[:,4],subset[:,4]))+5)/100.0)**e_t
        w = w_1*w_d + w_2*(1-w_d)
        inds_ = np.arange(num_shots**2)
        inds = np.arange(num_shots**2)[inds_%num_shots!=0]
        vals = np.squeeze(sigmoid(np.subtract.outer(subset[:,3],subset[:,3]),sig_p).reshape(-1,1))
        vals = vals[inds_%num_shots!=0]
        np.add.at(arr,inds,w*vals)
        np.add.at(arr1,inds,w*.5)
    mat,mat1 = csc_matrix(arr),csc_matrix(arr1)
    return (mat,mat1)

In [6]:
data.columns = [col.replace('#','') for col in data.columns]

In [7]:
cols = ('Year','Permanent_Tournament_#')
rawdata = pd.concat([pd.read_csv('../data/rawdata/hole/%d.txt' % year, sep=';', 
                                 usecols=lambda x: x.strip().replace(' ','_') in cols)
                     for year in range(2003,2018)])
tourn_order = rawdata.drop_duplicates().values.tolist()

In [9]:
with open('../PickleFiles/num_to_ind_shot.pkl','rb') as pickle_file:
    num_to_ind = pickle.load(pickle_file)

for player_num in data['Player_'].drop_duplicates():
    if player_num not in num_to_ind:
        num_to_ind[player_num] = len(num_to_ind)

with open('../PickleFiles/num_to_ind_shot.pkl','wb') as pickle_file:
    pickle.dump(num_to_ind,pickle_file)

data.insert(5,'Player_Index',[num_to_ind[num] for num in data.Player_])
n_players = len(num_to_ind)
print n_players
data.Time = data.Time.values/100 * 60 + data.Time.values%100

tourns_in_data = data[['Year','Permanent_Tournament_']].drop_duplicates().values.tolist()
tourns_in_data = set(tuple(tup) for tup in tourns_in_data)
tourn_order = [tup for tup in tourn_order if tuple(tup) in tourns_in_data]
tourn_seq = {tuple(tup):u for u,tup in enumerate(tourn_order)}
data['tourn_num'] = [tourn_seq[tuple(tup)] for tup in data[['Year','Permanent_Tournament_']].values]
n_tournaments = len(tourn_seq)

2048


In [13]:
condition = 'Cat=="Green" & Distance_from_hole<5'
e_d,e_t,w_d,p_mult = .8,.7,.8,1.3

In [15]:
def sigmoid(x,sig_p):
    m,r = sig_p, sig_p/10.
    return (1./(1. + np.exp(m)**(-x)) + (np.tanh(r*x) + 1.)/2.)/2.

In [99]:
arr,arr1 = np.zeros((n_players,n_players)),np.zeros((n_players,n_players))
for (round,course,hole),df in data[data.tourn_num==7].groupby(['Round','Course_','Hole']):
    subset = df.query(condition)[['Start_X_Coordinate','Start_Y_Coordinate','Distance_from_hole',
                                  'Strokes_Gained','Time','Player_Index']].values
    num_shots = subset.shape[0]
    dists = squareform(pdist(subset[:,0:2]))
    w_1 = w_1 = 1/(dists/(np.add.outer(subset[:,2],subset[:,2])/2) + .01)**e_d
    w_2 = 1/((np.abs(np.subtract.outer(subset[:,4],subset[:,4]))+5)/100.0)**e_t
    w = w_1*w_d + w_2*(1-w_d)
    np.fill_diagonal(w,0)
    w = np.squeeze(w.reshape(-1,1))
    vals = sigmoid(np.subtract.outer(subset[:,3],subset[:,3]),10.)
    np.fill_diagonal(vals,0)
    vals = np.squeeze(vals.reshape(-1,1))
    inds = (np.repeat(subset[:,5],num_shots).astype(int),np.tile(subset[:,5],num_shots).astype(int))
    np.add.at(arr,inds,w*vals)
    np.add.at(arr1,inds,w*.5)
mat,mat1 = csc_matrix(arr),csc_matrix(arr1)

In [97]:
np.squeeze(np.array([[3,4,2],[2,1,1]]).reshape(-1,1))

array([3, 4, 2, 2, 1, 1])

In [94]:
vals

array([ 0.        ,  0.49953008,  0.47372946,  0.5       ,  0.49794152,
        0.4998641 ,  0.49368972,  0.5       ,  0.48988414,  0.48733253,
        0.5       ,  0.49989074,  0.49966991,  0.4911875 ,  0.49793723,
        0.42773853,  0.49966991,  0.44688718,  0.49999499,  0.5       ,
        0.5       ,  0.49954706,  0.36904052,  0.4266752 ,  0.48889241,
        0.49962503,  0.43996722,  0.5       ,  0.49853767,  0.50046992,
        0.        ,  0.47419642,  0.50046992,  0.49841143,  0.50033402,
        0.49415948,  0.50046992,  0.49035364,  0.48780178,  0.50046992,
        0.50036067,  0.50013983,  0.4916571 ,  0.49840714,  0.42818583,
        0.50013983,  0.44734488,  0.50046492,  0.50046992,  0.50046992,
        0.50001699,  0.36943702,  0.42712183,  0.48936182,  0.50009495,
        0.44042152,  0.50046992,  0.49900759,  0.52627054,  0.52580358,
        0.        ,  0.52627054,  0.52422427,  0.52613551,  0.51999117,
        0.52627054,  0.51619482,  0.51364543,  0.52627054,  0.52

In [93]:
inds

(array([ 42,  42,  42,  42,  42,  42,  42,  42,  42,  42,  42,  42,  42,
         42,  42,  42,  42,  42,  42,  42,  42,  42,  42,  42,  42,  42,
         42,  42,  42,  49,  49,  49,  49,  49,  49,  49,  49,  49,  49,
         49,  49,  49,  49,  49,  49,  49,  49,  49,  49,  49,  49,  49,
         49,  49,  49,  49,  49,  49,  93,  93,  93,  93,  93,  93,  93,
         93,  93,  93,  93,  93,  93,  93,  93,  93,  93,  93,  93,  93,
         93,  93,  93,  93,  93,  93,  93,  93,  93, 103, 103, 103, 103,
        103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103,
        103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 114,
        114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114,
        114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114, 114,
        114, 114, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129,
        129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129,
        129, 129, 129, 129, 129, 174, 174, 174, 174

In [88]:
vals.shape

(784, 1)

In [86]:
vals.shape

(28, 28)

In [85]:
inds[1].shape

(784,)

In [84]:
inds[0].shape

(784,)

In [76]:
subset[0:5,5]

array([  49.,   90.,   93.,  103.,  114.])

In [79]:
(np.repeat(subset[0:5,5],5).shape,np.tile(subset[0:5,5],5).shape)

((25,), (25,))

(20,)

In [ ]:
[i for i in subset[:,5],subset[:,5]

In [60]:
np.diag_indices(10)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

In [61]:
np.where()

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9]),
 array([1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 3, 4, 5,
        6, 7, 8, 9, 0, 1, 2, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 5, 6, 7, 8, 9, 0,
        1, 2, 3, 4, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 7, 8, 9, 0, 1, 2, 3, 4, 5,
        6, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8]))

In [ ]:
np.di

In [57]:
a = np.zeros((10,10))

In [58]:
np.add.at(a,[[0,1],[2,4]],[3,2])

In [59]:
a

array([[ 0.,  0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [52]:
inds.shape

(756,)

In [51]:
(w*.5).shape

(756,)

In [46]:
inds.shape

(756,)

In [41]:
arr.shape

(2048, 2048)

In [42]:
vals

array([ 0.5420724 ,  0.49293634,  0.41469883,  0.54245784,  0.54089302,
        0.53997453,  0.54245784,  0.54072473,  0.43120666,  0.54245784,
        0.54244957,  0.53016223,  0.5182765 ,  0.47295838,  0.54017181,
        0.52899843,  0.53532776,  0.53969953,  0.53787723,  0.54244957,
        0.54198886,  0.5419572 ,  0.5208456 ,  0.54198886,  0.51917905,
        0.53969953,  0.48432447,  0.5       ,  0.45099953,  0.37669693,
        0.50039196,  0.49880142,  0.49786875,  0.50039196,  0.49863047,
        0.39202007,  0.50039196,  0.50038356,  0.48794731,  0.47603236,
        0.43160921,  0.49806901,  0.48677569,  0.49316065,  0.49758963,
        0.49574161,  0.50038356,  0.49991506,  0.49988288,  0.47859829,
        0.49991506,  0.4769332 ,  0.49758963,  0.4426043 ,  0.54900047,
        0.5       ,  0.42132352,  0.5493836 ,  0.54782804,  0.54691483,
        0.5493836 ,  0.54766072,  0.43799048,  0.5493836 ,  0.54937539,
        0.53715212,  0.52530969,  0.47998638,  0.54711098,  0.53

In [27]:
w.reshape(-1,1).shape

(756, 1)

In [30]:
(w*vals).shape

(756,)

In [24]:
w.shape

(756,)

In [17]:
vals.shape

(756,)

In [18]:
inds.shape

(756,)

In [19]:
arr.shape

(2048, 2048)

In [20]:
w.shape

(28, 28)